# LangGraph + MCP + Ollama Autonomous Agent

Plan → Human Approval → Act (Autonomous)


In [ ]:
!pip install -U langgraph langchain langchain-ollama mcp

In [ ]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_ollama import ChatOllama
import json, os

In [ ]:
llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0
)

## Agent State

In [ ]:
class AgentState(TypedDict):
    goal: str
    plan: List[str]
    approved: bool
    messages: List[Dict]
    done: bool

## MCP Filesystem Server (Sandboxed)

In [ ]:
from mcp.server.fastmcp import FastMCP

BASE_DIR = "/tmp/mcp_workspace"
os.makedirs(BASE_DIR, exist_ok=True)

mcp_server = FastMCP("filesystem")

@mcp_server.tool()
def list_dir(path: str = "."):
    return os.listdir(os.path.join(BASE_DIR, path))

@mcp_server.tool()
def read_file(path: str):
    return open(os.path.join(BASE_DIR, path)).read()

@mcp_server.tool()
def write_file(path: str, content: str):
    with open(os.path.join(BASE_DIR, path), "w") as f:
        f.write(content)
    return "ok"

## MCP Client

In [ ]:
from mcp import ClientSession
from mcp.server.stdio import stdio_server

server = stdio_server(mcp_server)
session = ClientSession(server.__enter__())
session.initialize()

## Planner Node (PLAN MODE)

In [ ]:
def planner_node(state: AgentState) -> AgentState:
    prompt = f"""
    You are an AI planner.

    Goal:
    {state['goal']}

    Create a high-level step-by-step plan.
    Do NOT execute anything.
    """

    plan_text = llm.invoke(prompt).content

    state["plan"] = [l for l in plan_text.splitlines() if l.strip()]
    state["approved"] = False
    state["messages"] = []
    state["done"] = False
    return state

## Executor Node (LLM decides tool)

In [ ]:
def executor_node(state: AgentState) -> AgentState:
    tools = session.list_tools()

    prompt = f"""
    You are an autonomous agent.

    Goal: {state['goal']}
    Plan: {state['plan']}
    History: {state['messages']}

    Available MCP tools:
    {tools}

    Decide ONE next action.

    Respond ONLY in JSON:
    {{
      \"tool\": \"<tool_name | done>\",
      \"args\": {{ }}
    }}
    """

    action = json.loads(llm.invoke(prompt).content)

    if action["tool"] == "done":
        state["done"] = True
        return state

    state["messages"].append(action)
    return state

## Tool Runner Node (MCP Dispatcher)

In [ ]:
def tool_node(state: AgentState) -> AgentState:
    action = state["messages"][-1]

    result = session.call_tool(
        action["tool"],
        action.get("args", {})
    )

    state["messages"].append({"observation": result})
    return state

## Build LangGraph

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("planner", planner_node)
graph.add_node("executor", executor_node)
graph.add_node("tool", tool_node)

graph.set_entry_point("planner")
graph.add_edge("planner", "WAIT")

graph.add_conditional_edges(
    "WAIT",
    lambda s: "executor" if s["approved"] else "WAIT"
)

graph.add_conditional_edges(
    "executor",
    lambda s: END if s["done"] else "tool"
)

graph.add_edge("tool", "executor")

app = graph.compile()

## Run PLAN MODE

In [ ]:
state = {
    "goal": "Read docs and write short notes into summary.txt",
    "plan": [],
    "approved": False,
    "messages": [],
    "done": False
}

state = app.invoke(state)
state["plan"]

## User clicks ACT

In [ ]:
state["approved"] = True
state = app.invoke(state)
state["messages"]